In [1]:
from numpy.core.numeric import True_
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
#from matplotlib import pyplot as plt
#from PIL import Image, ImageDraw
#from IPython import display
#from torchvision import transforms

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [3]:
data = 'default_credit_card'
df = pd.read_csv('../datasets/'+data+'_sc.csv')

In [4]:
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Default
0,0.010101,0.792236,0.762653,0.765284,0.051724,0.308763,0.443936,0.832081,0.826502,0.822306,...,0.160138,0.080648,0.260979,0.000000,0.000409,0.000000,0.000000,0.000000,0.000000,0
1,0.111111,0.792236,0.762653,0.790716,0.086207,0.843805,0.443936,0.825485,0.816710,0.811469,...,0.163220,0.084074,0.263485,0.000000,0.000594,0.001116,0.001610,0.000000,0.003783,0
2,0.080808,0.792236,0.762653,0.790716,0.224138,0.871881,0.840873,0.825485,0.816710,0.811469,...,0.173637,0.095470,0.272928,0.001738,0.000891,0.001116,0.001610,0.002345,0.009458,1
3,0.040404,0.792236,0.762653,0.765284,0.275862,0.871881,0.840873,0.825485,0.816710,0.811469,...,0.186809,0.109363,0.283685,0.002290,0.001199,0.001339,0.001771,0.002506,0.001892,1
4,0.040404,0.758329,0.762653,0.765284,0.620690,0.843805,0.840873,0.832081,0.816710,0.811469,...,0.179863,0.099633,0.275681,0.002290,0.021779,0.011160,0.014493,0.001615,0.001284,1


In [5]:
target = 'Default'
y = df[target].values
del df[target]
x = df.values

In [6]:
# train the classifier to be explained -> Logistic Regression Model (can be replaced with a MLP)
# clf = LogisticRegression(max_iter=1000, fit_intercept=False, class_weight='balanced')
# clf.fit(x, y)

In [7]:
# Convert numpy arrays to PyTorch tensors
x_train_tensor = torch.FloatTensor(x).to(device)
y_train_tensor = torch.LongTensor(y).to(device)

In [8]:
# Create a TensorDataset
dataset = TensorDataset(x_train_tensor, y_train_tensor)

In [9]:
# Create a DataLoader
dl_batch_size = 32  # Set your desired batch size
train_dataloader = DataLoader(dataset, batch_size=dl_batch_size, shuffle=True)

In [10]:
num_epochs = 500

In [11]:
x.shape[1]

23

In [12]:
latent_dims = 10
# capacity = no. of hidden neurons in a layer
capacity = 8
variational_beta = 1

In [13]:
latent_dims

10

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        c = capacity
        self.fc1 = nn.Linear(in_features=input_dim, out_features=c*2)
        self.fc2 = nn.Linear(in_features=c*2, out_features=latent_dims)
        self.fc3 = nn.Linear(in_features=c*2, out_features=latent_dims)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc2(x)
        x_logvar = self.fc3(x)
        return x_mu, x_logvar

class Decoder(nn.Module):
    def __init__(self, output_dim):
        super(Decoder, self).__init__()
        c = capacity
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*2)
        self.fc_output = nn.Linear(in_features=c*2, out_features=output_dim)
            
    def forward(self, x):
        x = F.relu(self.fc(x))
        x_recon = self.fc_output(x)
        return x_recon

class VariationalAutoencoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(input_dim)
        self.decoder = Decoder(output_dim)
    
    def forward(self, x):
        latent_mu, latent_logvar = self.encoder(x)
        latent = self.latent_sample(latent_mu, latent_logvar)
        x_recon = self.decoder(latent)
        return x_recon, latent_mu, latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')  # Mean Squared Error loss for tabular data
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + variational_beta * kldivergence

In [16]:
# Train VAE
print(device)
vae = VariationalAutoencoder(input_dim=x.shape[1], output_dim=x.shape[1])
vae = vae.to(device)

optimizer = torch.optim.Adam(params=vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_vae = True

if train_vae == False:   
    filename = 'blood_alcohol_vae.pth'
    vae.load_state_dict(torch.load(filename))
    vae.eval()
    print('done')

else:
    for epoch in range(num_epochs):
        vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for x_batch, _ in train_dataloader:

            x_batch = x_batch.to(device)

            # Forward pass
            x_recon, latent_mu, latent_logvar = vae(x_batch)
            loss = vae_loss(x_recon, x_batch, latent_mu, latent_logvar)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average loss: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_vae.pth'
    torch.save(vae.state_dict(), model_name)

mps
Epoch [1 / 500] average loss: 25.234815
Epoch [2 / 500] average loss: 5.752657
Epoch [3 / 500] average loss: 5.486117
Epoch [4 / 500] average loss: 5.466868
Epoch [5 / 500] average loss: 5.467518
Epoch [6 / 500] average loss: 5.462434
Epoch [7 / 500] average loss: 5.461699
Epoch [8 / 500] average loss: 5.458958
Epoch [9 / 500] average loss: 5.458442
Epoch [10 / 500] average loss: 5.459859
Epoch [11 / 500] average loss: 5.452501
Epoch [12 / 500] average loss: 5.457633
Epoch [13 / 500] average loss: 5.454558
Epoch [14 / 500] average loss: 5.453628
Epoch [15 / 500] average loss: 5.451424
Epoch [16 / 500] average loss: 5.453286
Epoch [17 / 500] average loss: 5.448668
Epoch [18 / 500] average loss: 5.451021
Epoch [19 / 500] average loss: 5.451856
Epoch [20 / 500] average loss: 5.450368
Epoch [21 / 500] average loss: 5.447157
Epoch [22 / 500] average loss: 5.448725
Epoch [23 / 500] average loss: 5.445492
Epoch [24 / 500] average loss: 5.446400
Epoch [25 / 500] average loss: 5.445077
Epoc

Epoch [204 / 500] average loss: 5.416816
Epoch [205 / 500] average loss: 5.416140
Epoch [206 / 500] average loss: 5.417375
Epoch [207 / 500] average loss: 5.416884
Epoch [208 / 500] average loss: 5.416441
Epoch [209 / 500] average loss: 5.417499
Epoch [210 / 500] average loss: 5.416527
Epoch [211 / 500] average loss: 5.417033
Epoch [212 / 500] average loss: 5.417079
Epoch [213 / 500] average loss: 5.417016
Epoch [214 / 500] average loss: 5.417592
Epoch [215 / 500] average loss: 5.417018
Epoch [216 / 500] average loss: 5.417480
Epoch [217 / 500] average loss: 5.416614
Epoch [218 / 500] average loss: 5.417321
Epoch [219 / 500] average loss: 5.417456
Epoch [220 / 500] average loss: 5.416778
Epoch [221 / 500] average loss: 5.416847
Epoch [222 / 500] average loss: 5.416171
Epoch [223 / 500] average loss: 5.416587
Epoch [224 / 500] average loss: 5.417589
Epoch [225 / 500] average loss: 5.417051
Epoch [226 / 500] average loss: 5.416513
Epoch [227 / 500] average loss: 5.416921
Epoch [228 / 500

Epoch [404 / 500] average loss: 5.417014
Epoch [405 / 500] average loss: 5.417430
Epoch [406 / 500] average loss: 5.416824
Epoch [407 / 500] average loss: 5.416920
Epoch [408 / 500] average loss: 5.416906
Epoch [409 / 500] average loss: 5.417099
Epoch [410 / 500] average loss: 5.416586
Epoch [411 / 500] average loss: 5.416802
Epoch [412 / 500] average loss: 5.417181
Epoch [413 / 500] average loss: 5.417757
Epoch [414 / 500] average loss: 5.417091
Epoch [415 / 500] average loss: 5.417251
Epoch [416 / 500] average loss: 5.417070
Epoch [417 / 500] average loss: 5.416653
Epoch [418 / 500] average loss: 5.417220
Epoch [419 / 500] average loss: 5.416905
Epoch [420 / 500] average loss: 5.417425
Epoch [421 / 500] average loss: 5.417067
Epoch [422 / 500] average loss: 5.417194
Epoch [423 / 500] average loss: 5.417220
Epoch [424 / 500] average loss: 5.417105
Epoch [425 / 500] average loss: 5.417218
Epoch [426 / 500] average loss: 5.417616
Epoch [427 / 500] average loss: 5.416995
Epoch [428 / 500

In [17]:
c2c_latent_dims = latent_dims-1
print(c2c_latent_dims)
c2c_capacity = latent_dims
print(c2c_capacity)
c2c_variational_beta = 1
label_size = 2  # Assuming two classes

9
10


In [18]:
class c2c_Encoder(nn.Module):
    def __init__(self):
        super(c2c_Encoder, self).__init__()
        self.fc1 = nn.Linear(in_features=latent_dims + label_size, out_features=c2c_capacity)
        self.fc_mu = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
        self.fc_logvar = nn.Linear(in_features=c2c_capacity, out_features=c2c_latent_dims)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x_mu = self.fc_mu(x)
        x_logvar = self.fc_logvar(x)
        return x_mu, x_logvar

class c2c_Decoder(nn.Module):
    def __init__(self):
        super(c2c_Decoder, self).__init__()
        self.fc2 = nn.Linear(in_features=c2c_latent_dims + label_size, out_features=c2c_capacity)
        self.fc1 = nn.Linear(in_features=c2c_capacity, out_features=latent_dims)
            
    def forward(self, x):
        x = self.fc2(x)
        x = self.fc1(x)
        return x
    
class c2c_VariationalAutoencoder(nn.Module):
    def __init__(self):
        super(c2c_VariationalAutoencoder, self).__init__()
        self.encoder = c2c_Encoder()
        self.decoder = c2c_Decoder()
    
    def forward(self, x):
        x,label = torch.split(x,(latent_dims,label_size),dim = 1)
        c2c_latent_mu, c2c_latent_logvar = self.encoder(torch.cat((x,label), dim=1))
        c2c_latent = self.latent_sample(c2c_latent_mu, c2c_latent_logvar)
        x_recon = self.decoder(torch.cat((c2c_latent,label), dim=1))
        return x_recon, c2c_latent_mu, c2c_latent_logvar
    
    def latent_sample(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    
def c2c_vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x.view(-1, latent_dims), x.view(-1, latent_dims), reduction='sum')
    
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + c2c_variational_beta * kldivergence

In [19]:
print(device)
c2c_vae = c2c_VariationalAutoencoder()
c2c_vae = c2c_vae.to(device)

optimizer = torch.optim.Adam(params=c2c_vae.parameters(), lr=1e-3, weight_decay=1e-5)

train_loss_avg = []

train_c2c_vae = True

if train_c2c_vae == False:
    filename = 'blood_alcohol_c2c.pth'
    c2c_vae.load_state_dict(torch.load(filename))
    c2c_vae.eval()
    print('done')
else:
    for epoch in range(num_epochs):
        c2c_vae.train()
        train_loss_avg.append(0)
        num_batches = 0

        for i in range(dl_batch_size):

            x_batch1, labels1 = next(iter(train_dataloader))
            x_batch2, labels2 = next(iter(train_dataloader))

            x_batch1 = x_batch1.to(device)
            x_batch_recon1, latent_mu1, latent_logvar1 = vae(x_batch1)

            x_batch2 = x_batch2.to(device)
            x_batch_recon2, latent_mu2, latent_logvar2 = vae(x_batch2) 

            latent_diff = latent_mu1 - latent_mu2 
            label_diff = torch.cat((labels1.unsqueeze(1), labels2.unsqueeze(1)), dim=1)
            #print(label_diff)

            latent_diff = latent_diff.to(device)
            label_diff = label_diff.float().to(device)

            c2c_input = torch.cat((latent_diff,label_diff), dim=1)
            c2c_input = c2c_input.to(device)

            c2c_recon, c2c_latent_mu, c2c_latent_logvar = c2c_vae(c2c_input)
            loss = c2c_vae_loss(c2c_recon, latent_diff.detach(), c2c_latent_mu, c2c_latent_logvar)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            train_loss_avg[-1] += loss.item()
            num_batches += 1

        train_loss_avg[-1] /= num_batches
        print('Epoch [%d / %d] average reconstruction error: %f' % (epoch + 1, num_epochs, train_loss_avg[-1]))

    # Save the model
    model_name = data+'_c2c.pth'
    torch.save(c2c_vae.state_dict(), model_name)

mps
Epoch [1 / 500] average reconstruction error: 51.397806
Epoch [2 / 500] average reconstruction error: 25.695095
Epoch [3 / 500] average reconstruction error: 13.620853
Epoch [4 / 500] average reconstruction error: 7.784403
Epoch [5 / 500] average reconstruction error: 4.572469
Epoch [6 / 500] average reconstruction error: 2.778016
Epoch [7 / 500] average reconstruction error: 1.777586
Epoch [8 / 500] average reconstruction error: 1.246149
Epoch [9 / 500] average reconstruction error: 0.902841
Epoch [10 / 500] average reconstruction error: 0.701416
Epoch [11 / 500] average reconstruction error: 0.591714
Epoch [12 / 500] average reconstruction error: 0.496144
Epoch [13 / 500] average reconstruction error: 0.421491
Epoch [14 / 500] average reconstruction error: 0.351383
Epoch [15 / 500] average reconstruction error: 0.305379
Epoch [16 / 500] average reconstruction error: 0.259107
Epoch [17 / 500] average reconstruction error: 0.210981
Epoch [18 / 500] average reconstruction error: 0.1

Epoch [146 / 500] average reconstruction error: -0.000007
Epoch [147 / 500] average reconstruction error: -0.000007
Epoch [148 / 500] average reconstruction error: -0.000007
Epoch [149 / 500] average reconstruction error: -0.000007
Epoch [150 / 500] average reconstruction error: -0.000007
Epoch [151 / 500] average reconstruction error: -0.000007
Epoch [152 / 500] average reconstruction error: -0.000007
Epoch [153 / 500] average reconstruction error: -0.000008
Epoch [154 / 500] average reconstruction error: -0.000007
Epoch [155 / 500] average reconstruction error: -0.000007
Epoch [156 / 500] average reconstruction error: -0.000006
Epoch [157 / 500] average reconstruction error: -0.000007
Epoch [158 / 500] average reconstruction error: -0.000007
Epoch [159 / 500] average reconstruction error: -0.000007
Epoch [160 / 500] average reconstruction error: -0.000007
Epoch [161 / 500] average reconstruction error: -0.000007
Epoch [162 / 500] average reconstruction error: -0.000007
Epoch [163 / 5

Epoch [288 / 500] average reconstruction error: -0.000007
Epoch [289 / 500] average reconstruction error: -0.000007
Epoch [290 / 500] average reconstruction error: -0.000007
Epoch [291 / 500] average reconstruction error: -0.000006
Epoch [292 / 500] average reconstruction error: -0.000007
Epoch [293 / 500] average reconstruction error: -0.000007
Epoch [294 / 500] average reconstruction error: -0.000007
Epoch [295 / 500] average reconstruction error: -0.000007
Epoch [296 / 500] average reconstruction error: -0.000006
Epoch [297 / 500] average reconstruction error: -0.000006
Epoch [298 / 500] average reconstruction error: -0.000007
Epoch [299 / 500] average reconstruction error: -0.000007
Epoch [300 / 500] average reconstruction error: -0.000007
Epoch [301 / 500] average reconstruction error: -0.000006
Epoch [302 / 500] average reconstruction error: -0.000007
Epoch [303 / 500] average reconstruction error: -0.000007
Epoch [304 / 500] average reconstruction error: -0.000007
Epoch [305 / 5

Epoch [430 / 500] average reconstruction error: 0.000072
Epoch [431 / 500] average reconstruction error: 0.000242
Epoch [432 / 500] average reconstruction error: 0.001801
Epoch [433 / 500] average reconstruction error: 0.001881
Epoch [434 / 500] average reconstruction error: 0.000897
Epoch [435 / 500] average reconstruction error: 0.000259
Epoch [436 / 500] average reconstruction error: 0.000126
Epoch [437 / 500] average reconstruction error: 0.000049
Epoch [438 / 500] average reconstruction error: 0.000022
Epoch [439 / 500] average reconstruction error: 0.000003
Epoch [440 / 500] average reconstruction error: 0.000001
Epoch [441 / 500] average reconstruction error: -0.000005
Epoch [442 / 500] average reconstruction error: -0.000005
Epoch [443 / 500] average reconstruction error: -0.000001
Epoch [444 / 500] average reconstruction error: -0.000005
Epoch [445 / 500] average reconstruction error: -0.000006
Epoch [446 / 500] average reconstruction error: -0.000006
Epoch [447 / 500] average